**PREPARE FOR ML**

Import libraries

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk import WordNetLemmatizer, pos_tag, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

Import csv

In [ ]:
# Define function to read csv file into dataset
def read_csv_file(file): 
    dataset = pd.read_csv(file,header=0)
    return dataset

# Call funtion on file
#df = read_csv_file(r"C:\Users\nayma\AppData\Local\Programs\Python\Python312\Scripts\thesis\df_polarity.csv")
path = r"C:\Users\jmarsal\AppData\Local\anaconda3\Scripts\thesis\df_train.csv"
df = read_csv_file(path)

# Work only with relevant columns
df = df[['post_id','comment_id','comment_text','polarity']]
df.head()

Remove stop words

In [ ]:
# List of stop words
stop_words = list(stopwords.words('english'))

# List of missing stop words
missing_stop_words = ["i'm", "I'm", 'im', 'Im','u', 'ur','youre','yall','yalls','the',"don't",'dont','doesnt']

# Define functions to uppercase list and merge
def up_and_merge(x):
    x = merge_list(x, uppercase_list(x))
    return x
    
def merge_list(x,*y):
    for z in y:
        x = x + z
    return x

def uppercase_list(x):
    x_up = [i.upper() for i in x]
    return x_up

stop_words.extend(missing_stop_words)
stop_words = up_and_merge(stop_words)
print(stop_words)

In [ ]:
# Define function to remove stopwords
def remove_words(text_data:str,stop_words):
    # Remove all words as specified in a custom list of words
    return [item for item in text_data if item not in stop_words]

# Define function to collapse to string
def collapse_list_to_string(string_list):
    # This is to join back together the text data into a single string
    return ' '.join(string_list)


# Remove stop words
df['comment_train'] = df['comment_text'].astype(str).apply(lambda x: remove_words(x.split(),stop_words))
df['comment_train'] = df['comment_train'].apply(collapse_list_to_string)

Remove irrelevant punctuation

In [ ]:
# Remove irrelevant pucntuation and numbers, keep *!
def remove_irrelevant_punctuation(text):
    list_punct = ["'", ":", ")", "(", '"', '`', '/', ".", "?", "@", "#", "$", "%", "^", "&", "+", "_", "~", ",", "’", 
                  " 🏾", "<", ">", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for i in list_punct:
        text = text.replace(i, "")                
    return text

# Add space between words and ! 
def add_space(comment):
    comment = ''.join(' ' + char + ' ' if char=="!" else char for char in comment).strip()
    return comment

def blank_spaces(comment):
    comment = comment.replace(r' {2,}',' ')
    comment = comment.strip()
    return comment
    
    
df['comment_train'] = df['comment_train'].apply(remove_irrelevant_punctuation)
df['comment_train'] = df['comment_train'].apply(add_space)
df['comment_train'] = df['comment_train'].apply(blank_spaces)

Lemmatize (note this may not do uppers)

In [ ]:
lemmatizer = WordNetLemmatizer()

# Define function to tag part-of-speech word if they are verbs or adjectives
def pos(word):
    pos_label = (pos_tag(word_tokenize(word))[0][1][0]).lower()
    if pos_label == 'j': pos_label = 'a'  
    if pos_label not in ['a', 's', 'v']: #if word is not an adjective or a verb, label as a noun
        pos_label = 'n'
    return pos_label

# Define function to lemmatize words
def lemm(text):
    lemma = []
    for t in text.split():
        if t.isupper():
            t = lemmatizer.lemmatize(t.lower(), pos(t.lower()))
            lemma.append(t.upper())
        else:
            t = lemmatizer.lemmatize(t, pos(t))
            lemma.append(t)
    text = ' '.join(lemma)
    return text
        
# Lemmatize words in comments
df['comment_train'] = df['comment_train'].apply(lemm)

Save dataset to work on further notebooks

In [ ]:
path = r"C:\Users\jmarsal\AppData\Local\anaconda3\Scripts\thesis\df_to_train.csv"
df.to_csv(path, index=False)